In [1]:
import torch
import pandas as pd
import numpy as np


In [2]:
dataset_cols = ['id','text','category']
df= pd.read_csv('/content/smileannotationsfinal.csv',
                names = dataset_cols)
df.head()

,id,text,category
0,611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [3]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [4]:
df = df[df['category']!='nocode']
df = df[df['category']!='happy|surprise']
df = df[df['category']!='happy|sad']
df = df[df['category']!='sad|angry'] 
df = df[df['category']!='disgust|angry']
df = df[df['category']!='sad|disgust']
df = df[df['category']!='sad|disgust|angry']
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [5]:
labels= df.category.unique()
label_dict={}
for i,label in enumerate(labels,0):
    label_dict[label]=i
print(label_dict)
df['label']=df.category.replace(label_dict)
df.head(10)

{'happy': 0, 'not-relevant': 1, 'angry': 2, 'disgust': 3, 'sad': 4, 'surprise': 5}


,id,text,category,label
1,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
2,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
3,614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
4,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
5,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
9,614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
12,613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
15,613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
16,610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1
18,612648200588038144,@BarbyWT @britishmuseum so beautiful,happy,0


In [27]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=17,
                                                  stratify = df.label.values)
print(len(x_train),len(x_val))

1258 223


In [28]:
df['set_type']='nil'*df.shape[0]
df.loc[x_train,'set_type']= 'train'
df.loc[x_val,'set_type']= 'val'
df.groupby(['category','label','set_type']).count()

id  text
category     label set_type           
angry        2     train      48    48
                   val         9     9
disgust      3     train       5     5
                   val         1     1
happy        0     train     966   966
                   val       171   171
not-relevant 1     train     182   182
                   val        32    32
sad          4     train      27    27
                   val         5     5
surprise     5     train      30    30
                   val         5     5

In [8]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased',
                                         do_lower_case=True)

In [29]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.set_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.set_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.set_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.set_type=='val'].label.values)


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [30]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [11]:
from transformers import BertForSequenceClassification

In [12]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                     num_labels = len(label_dict),
                                     output_attentions = False,
                                     output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
dataloader_train = DataLoader(
    dataset_train,
    sampler= RandomSampler(dataset_train),
    batch_size=32
)
dataloader_val = DataLoader(
    dataset_val,
    sampler = SequentialSampler(dataset_val),
    batch_size=32
)

In [14]:
from transformers import AdamW, get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(),
                 lr=1e-5,
                 eps=1e-8)
epochs =10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

In [15]:
from sklearn.metrics import f1_score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis =1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')
def accuracy_per_class(preds, labels):
    label_dic_inverse = {v: k for k,v in label_dict.items()}
    preds_flat = np.argmax(preds, axis =1).flatten()
    labels_flat = labels.flatten()
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dic_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

In [16]:
import random

seed_val = 10
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print(device)


cuda


In [18]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [24]:
from tqdm.notebook import tqdm
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    training_loss=0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False
                       )
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        outputs = model(**inputs)
        loss=outputs[0]
        training_loss+= loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = training_loss/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation Loss: {val_loss}')
    tqdm.write(f'F1 score: {val_f1}')
    
        


Epoch 1
Training loss: 0.24485374186187983
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 2
Training loss: 0.2406100148335099
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 3
Training loss: 0.23035040348768235
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 4
Training loss: 0.24318218510597944
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 5
Training loss: 0.23950275182723998
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 6
Training loss: 0.2433488169685006
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 7
Training loss: 0.2374256355687976
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 8
Training loss: 0.23705278150737286
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 9
Training loss: 0.23880912009626626
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



Epoch 10
Training loss: 0.23929821541532875
Validation Loss: 0.4059011659451893
F1 score: 0.8593295506990719



In [32]:
loss, predictions, true_vals = evaluate(dataloader_val)

In [33]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 170/171
Class: not-relevant
Accuracy: 27/32
Class: angry
Accuracy: 8/9
Class: disgust
Accuracy: 0/1
Class: sad
Accuracy: 0/5
Class: surprise
Accuracy: 4/5
